Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    #here [1,2,2,1] shows the strides in various dimensions 2,2 means stride = 2 in height and width
    
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.987561
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 100: 1.238074
Minibatch accuracy: 56.2%
Validation accuracy: 66.5%
Minibatch loss at step 200: 0.931941
Minibatch accuracy: 75.0%
Validation accuracy: 78.3%
Minibatch loss at step 300: 0.535495
Minibatch accuracy: 81.2%
Validation accuracy: 78.5%
Minibatch loss at step 400: 0.385400
Minibatch accuracy: 87.5%
Validation accuracy: 79.8%
Minibatch loss at step 500: 0.817267
Minibatch accuracy: 75.0%
Validation accuracy: 81.2%
Minibatch loss at step 600: 0.534624
Minibatch accuracy: 87.5%
Validation accuracy: 81.0%
Minibatch loss at step 700: 0.898024
Minibatch accuracy: 75.0%
Validation accuracy: 82.2%
Minibatch loss at step 800: 0.871615
Minibatch accuracy: 62.5%
Validation accuracy: 82.7%
Minibatch loss at step 900: 0.579748
Minibatch accuracy: 81.2%
Validation accuracy: 82.8%
Minibatch loss at step 1000: 0.371478
Minibatch accuracy: 87.5%
Validation accuracy: 82.8%

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
#pooling after relu as per CS231n http://cs231n.github.io/convolutional-networks/

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1)) #16 filters/kernels here
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden])) #no conv at this time
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))                          #affine layer
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME') #no strides as per question
    
    hidden = tf.nn.relu(conv + layer1_biases)
    #printing these values to understand the operations 
    print (hidden.get_shape().as_list())
    hidden = tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding = 'SAME')
    print (hidden.get_shape().as_list())
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    print (hidden.get_shape().as_list())
    hidden = tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding = 'SAME')
    shape = hidden.get_shape().as_list()
    print (shape)
    
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print (reshape.get_shape().as_list())
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print (hidden.get_shape().as_list())
    print ("yes")
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

[16, 28, 28, 16]
[16, 14, 14, 16]
[16, 14, 14, 16]
[16, 7, 7, 16]
[16, 784]
[16, 64]
yes
[10000, 28, 28, 16]
[10000, 14, 14, 16]
[10000, 14, 14, 16]
[10000, 7, 7, 16]
[10000, 784]
[10000, 64]
yes
[10000, 28, 28, 16]
[10000, 14, 14, 16]
[10000, 14, 14, 16]
[10000, 7, 7, 16]
[10000, 784]
[10000, 64]
yes


In [8]:
train_prediction.get_shape().as_list()

[16, 10]

In [9]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))

  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.639127
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 100: 1.678551
Minibatch accuracy: 37.5%
Validation accuracy: 56.8%
Minibatch loss at step 200: 1.114138
Minibatch accuracy: 75.0%
Validation accuracy: 75.9%
Minibatch loss at step 300: 0.457752
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Minibatch loss at step 400: 0.354129
Minibatch accuracy: 87.5%
Validation accuracy: 79.7%
Minibatch loss at step 500: 0.760008
Minibatch accuracy: 87.5%
Validation accuracy: 81.3%
Minibatch loss at step 600: 0.381207
Minibatch accuracy: 87.5%
Validation accuracy: 81.4%
Minibatch loss at step 700: 1.000880
Minibatch accuracy: 75.0%
Validation accuracy: 83.0%
Minibatch loss at step 800: 0.642745
Minibatch accuracy: 81.2%
Validation accuracy: 83.7%
Minibatch loss at step 900: 0.648222
Minibatch accuracy: 81.2%
Validation accuracy: 83.4%
Minibatch loss at step 1000: 0.337172
Minibatch accuracy: 93.8%
Validation accuracy: 83.6%

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

<pre> 
Let us try using average pooling with no dropout for now. We will increase the number of steps 
</pre>

In [10]:
#pooling after relu as per CS231n http://cs231n.github.io/convolutional-networks/

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1)) #16 filters/kernels here
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden])) #no conv at this time
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))                          #affine layer
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME') #no strides as per question
    
    hidden = tf.nn.relu(conv + layer1_biases)
    #printing these values to understand the operations 
    print (hidden.get_shape().as_list())
    hidden = tf.nn.avg_pool(hidden,[1,2,2,1],[1,2,2,1],padding = 'SAME')
    print (hidden.get_shape().as_list())
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    print (hidden.get_shape().as_list())
    hidden = tf.nn.avg_pool(hidden,[1,2,2,1],[1,2,2,1],padding = 'SAME')
    shape = hidden.get_shape().as_list()
    print (shape)
    
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print (reshape.get_shape().as_list())
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print (hidden.get_shape().as_list())
    print ("yes")
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

[16, 28, 28, 16]
[16, 14, 14, 16]
[16, 14, 14, 16]
[16, 7, 7, 16]
[16, 784]
[16, 64]
yes
[10000, 28, 28, 16]
[10000, 14, 14, 16]
[10000, 14, 14, 16]
[10000, 7, 7, 16]
[10000, 784]
[10000, 64]
yes
[10000, 28, 28, 16]
[10000, 14, 14, 16]
[10000, 14, 14, 16]
[10000, 7, 7, 16]
[10000, 784]
[10000, 64]
yes


In [11]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))

  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.106863
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 0.320287
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
Minibatch loss at step 2000: 0.131710
Minibatch accuracy: 100.0%
Validation accuracy: 84.5%
Minibatch loss at step 3000: 0.852130
Minibatch accuracy: 81.2%
Validation accuracy: 85.7%
Minibatch loss at step 4000: 0.305434
Minibatch accuracy: 93.8%
Validation accuracy: 86.9%
Minibatch loss at step 5000: 1.102510
Minibatch accuracy: 68.8%
Validation accuracy: 86.9%
Minibatch loss at step 6000: 0.075902
Minibatch accuracy: 100.0%
Validation accuracy: 87.2%
Minibatch loss at step 7000: 0.485459
Minibatch accuracy: 75.0%
Validation accuracy: 87.9%
Minibatch loss at step 8000: 0.651363
Minibatch accuracy: 81.2%
Validation accuracy: 88.2%
Minibatch loss at step 9000: 0.106011
Minibatch accuracy: 100.0%
Validation accuracy: 88.5%
Minibatch loss at step 10000: 0.192576
Minibatch accuracy: 93.8%
Validation acc

<pre>
Great 94.4 % accuracy lets build up on top of it and apply dropout
</pre>

In [15]:
#pooling after relu as per CS231n http://cs231n.github.io/convolutional-networks/

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1)) #16 filters/kernels here
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden])) #no conv at this time
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))                          #affine layer
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  global_step = tf.Variable(0,name = 'global_step')
    
  # Model.
  def model(data,prob = 1.0):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME') #no strides as per question
    
    hidden = tf.nn.relu(conv + layer1_biases)
    #printing these values to understand the operations 
    print (hidden.get_shape().as_list())
    hidden = tf.nn.avg_pool(hidden,[1,2,2,1],[1,2,2,1],padding = 'SAME')
    
    hidden = tf.nn.dropout(hidden,keep_prob = prob) #0.6 was successful with fully connected network
    print (hidden.get_shape().as_list())
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    print (hidden.get_shape().as_list())
    hidden = tf.nn.avg_pool(hidden,[1,2,2,1],[1,2,2,1],padding = 'SAME')
    hidden = tf.nn.dropout(hidden,keep_prob = prob)
    shape = hidden.get_shape().as_list()
    print (shape)
    
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print (reshape.get_shape().as_list())
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print (hidden.get_shape().as_list())
    print ("yes")
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset,prob = 0.6)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

[16, 28, 28, 16]
[16, 14, 14, 16]
[16, 14, 14, 16]
[16, 7, 7, 16]
[16, 784]
[16, 64]
yes
[10000, 28, 28, 16]
[10000, 14, 14, 16]
[10000, 14, 14, 16]
[10000, 7, 7, 16]
[10000, 784]
[10000, 64]
yes
[10000, 28, 28, 16]
[10000, 14, 14, 16]
[10000, 14, 14, 16]
[10000, 7, 7, 16]
[10000, 784]
[10000, 64]
yes


In [16]:
num_steps = 10001 #increasing it will see how is the performance as learning rate is decaying we need more steps 

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))

  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.172837
Minibatch accuracy: 18.8%
Validation accuracy: 9.9%
Minibatch loss at step 1000: 0.411054
Minibatch accuracy: 87.5%
Validation accuracy: 82.2%
Minibatch loss at step 2000: 0.335139
Minibatch accuracy: 87.5%
Validation accuracy: 84.3%
Minibatch loss at step 3000: 0.849488
Minibatch accuracy: 81.2%
Validation accuracy: 85.7%
Minibatch loss at step 4000: 0.366918
Minibatch accuracy: 87.5%
Validation accuracy: 86.2%
Minibatch loss at step 5000: 0.813655
Minibatch accuracy: 75.0%
Validation accuracy: 86.5%
Minibatch loss at step 6000: 0.190220
Minibatch accuracy: 100.0%
Validation accuracy: 87.0%
Minibatch loss at step 7000: 0.643005
Minibatch accuracy: 75.0%
Validation accuracy: 87.0%
Minibatch loss at step 8000: 0.570708
Minibatch accuracy: 87.5%
Validation accuracy: 86.8%
Minibatch loss at step 9000: 0.209786
Minibatch accuracy: 93.8%
Validation accuracy: 87.8%
Minibatch loss at step 10000: 0.123231
Minibatch accuracy: 100.0%
Validation accu

<pre> A little less than previous lets do decaying rate this time without dropout</pre>

In [17]:
#We will put prob = 1.0 so to specify no dropout

#pooling after relu as per CS231n http://cs231n.github.io/convolutional-networks/

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1)) #16 filters/kernels here
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden])) #no conv at this time
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))                          #affine layer
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  global_step = tf.Variable(0,name = 'global_step')
    
  # Model.
  def model(data,prob = 1.0):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME') #no strides as per question
    
    hidden = tf.nn.relu(conv + layer1_biases)
    #printing these values to understand the operations 
    print (hidden.get_shape().as_list())
    hidden = tf.nn.avg_pool(hidden,[1,2,2,1],[1,2,2,1],padding = 'SAME')
    
    hidden = tf.nn.dropout(hidden,keep_prob = prob) #0.6 was successful with fully connected network
    print (hidden.get_shape().as_list())
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    print (hidden.get_shape().as_list())
    hidden = tf.nn.avg_pool(hidden,[1,2,2,1],[1,2,2,1],padding = 'SAME')
    hidden = tf.nn.dropout(hidden,keep_prob = prob)
    shape = hidden.get_shape().as_list()
    print (shape)
    
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print (reshape.get_shape().as_list())
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print (hidden.get_shape().as_list())
    print ("yes")
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset,prob = 1.0)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  learning_rate = tf.train.exponential_decay(0.05,global_step,1000,0.70,staircase=True)
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step = global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 10001 #increasing it will see how is the performance as learning rate is decaying we need more steps 

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))

  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

[16, 28, 28, 16]
[16, 14, 14, 16]
[16, 14, 14, 16]
[16, 7, 7, 16]
[16, 784]
[16, 64]
yes
[10000, 28, 28, 16]
[10000, 14, 14, 16]
[10000, 14, 14, 16]
[10000, 7, 7, 16]
[10000, 784]
[10000, 64]
yes
[10000, 28, 28, 16]
[10000, 14, 14, 16]
[10000, 14, 14, 16]
[10000, 7, 7, 16]
[10000, 784]
[10000, 64]
yes
Initialized
Minibatch loss at step 0: 3.371956
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 2000: 0.214467
Minibatch accuracy: 93.8%
Validation accuracy: 84.7%
Minibatch loss at step 4000: 0.297199
Minibatch accuracy: 87.5%
Validation accuracy: 86.5%
Minibatch loss at step 6000: 0.177374
Minibatch accuracy: 93.8%
Validation accuracy: 86.7%
Minibatch loss at step 8000: 0.597817
Minibatch accuracy: 81.2%
Validation accuracy: 87.2%
Minibatch loss at step 10000: 0.082074
Minibatch accuracy: 100.0%
Validation accuracy: 87.1%
Test accuracy: 94.0%


<pre>
No improvement finally lets try dropout and decaying 
</pre>

In [18]:
#We will put prob = 1.0 so to specify no dropout

#pooling after relu as per CS231n http://cs231n.github.io/convolutional-networks/

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1)) #16 filters/kernels here
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden])) #no conv at this time
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))                          #affine layer
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  global_step = tf.Variable(0,name = 'global_step')
    
  # Model.
  def model(data,prob = 1.0):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME') #no strides as per question
    
    hidden = tf.nn.relu(conv + layer1_biases)
    #printing these values to understand the operations 
    print (hidden.get_shape().as_list())
    hidden = tf.nn.avg_pool(hidden,[1,2,2,1],[1,2,2,1],padding = 'SAME')
    
    hidden = tf.nn.dropout(hidden,keep_prob = prob) #0.6 was successful with fully connected network
    print (hidden.get_shape().as_list())
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    print (hidden.get_shape().as_list())
    hidden = tf.nn.avg_pool(hidden,[1,2,2,1],[1,2,2,1],padding = 'SAME')
    hidden = tf.nn.dropout(hidden,keep_prob = prob)
    shape = hidden.get_shape().as_list()
    print (shape)
    
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print (reshape.get_shape().as_list())
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print (hidden.get_shape().as_list())
    print ("yes")
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset,prob = 0.6)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  learning_rate = tf.train.exponential_decay(0.05,global_step,1000,0.70,staircase=True)
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step = global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 10001 #increasing it will see how is the performance as learning rate is decaying we need more steps 

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))

  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

[16, 28, 28, 16]
[16, 14, 14, 16]
[16, 14, 14, 16]
[16, 7, 7, 16]
[16, 784]
[16, 64]
yes
[10000, 28, 28, 16]
[10000, 14, 14, 16]
[10000, 14, 14, 16]
[10000, 7, 7, 16]
[10000, 784]
[10000, 64]
yes
[10000, 28, 28, 16]
[10000, 14, 14, 16]
[10000, 14, 14, 16]
[10000, 7, 7, 16]
[10000, 784]
[10000, 64]
yes
Initialized
Minibatch loss at step 0: 3.512746
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 2000: 0.235962
Minibatch accuracy: 93.8%
Validation accuracy: 84.2%
Minibatch loss at step 4000: 0.391391
Minibatch accuracy: 81.2%
Validation accuracy: 85.6%
Minibatch loss at step 6000: 0.214686
Minibatch accuracy: 87.5%
Validation accuracy: 86.3%
Minibatch loss at step 8000: 0.598588
Minibatch accuracy: 81.2%
Validation accuracy: 86.4%
Minibatch loss at step 10000: 0.133426
Minibatch accuracy: 93.8%
Validation accuracy: 86.4%
Test accuracy: 92.7%


<pre> So a little bad so the best that is achieved is 94% </pre>

<h3> One more convolution layer </h3>

In [21]:
#pooling after relu as per CS231n http://cs231n.github.io/convolutional-networks/

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1)) #16 filters/kernels here
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  layer4_weights = tf.Variable(tf.truncated_normal(
      [256, num_hidden], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden])) #no conv at this time
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))                          #affine layer
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME') #no strides as per question
    
    hidden = tf.nn.relu(conv + layer1_biases)
    #printing these values to understand the operations 
    print (hidden.get_shape().as_list())
    hidden = tf.nn.avg_pool(hidden,[1,2,2,1],[1,2,2,1],padding = 'SAME')
    print (hidden.get_shape().as_list())
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    print (hidden.get_shape().as_list())
    hidden = tf.nn.avg_pool(hidden,[1,2,2,1],[1,2,2,1],padding = 'SAME')
    shape = hidden.get_shape().as_list()
    print (shape)
    
    conv = tf.nn.conv2d(hidden, layer3_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer3_biases)
    print (hidden.get_shape().as_list())
    hidden = tf.nn.avg_pool(hidden,[1,2,2,1],[1,2,2,1],padding = 'SAME')
    shape = hidden.get_shape().as_list()
    print (shape)
    print ("yes")
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print (reshape.get_shape().as_list())
    hidden = tf.nn.relu(tf.matmul(reshape, layer4_weights) + layer4_biases)
    print (hidden.get_shape().as_list())
    print ("yes")
    return tf.matmul(hidden, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

[16, 28, 28, 16]
[16, 14, 14, 16]
[16, 14, 14, 16]
[16, 7, 7, 16]
[16, 7, 7, 16]
[16, 4, 4, 16]
yes
[16, 256]
[16, 64]
yes
[10000, 28, 28, 16]
[10000, 14, 14, 16]
[10000, 14, 14, 16]
[10000, 7, 7, 16]
[10000, 7, 7, 16]
[10000, 4, 4, 16]
yes
[10000, 256]
[10000, 64]
yes
[10000, 28, 28, 16]
[10000, 14, 14, 16]
[10000, 14, 14, 16]
[10000, 7, 7, 16]
[10000, 7, 7, 16]
[10000, 4, 4, 16]
yes
[10000, 256]
[10000, 64]
yes


In [24]:
num_steps = 32001 #training for more time 95.2 % for 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 4000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))

  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.619383
Minibatch accuracy: 12.5%
Validation accuracy: 7.7%
Minibatch loss at step 4000: 0.331697
Minibatch accuracy: 87.5%
Validation accuracy: 86.3%
Minibatch loss at step 8000: 0.546938
Minibatch accuracy: 81.2%
Validation accuracy: 88.1%
Minibatch loss at step 12000: 0.425303
Minibatch accuracy: 81.2%
Validation accuracy: 88.9%
Minibatch loss at step 16000: 0.913112
Minibatch accuracy: 81.2%
Validation accuracy: 89.5%
Minibatch loss at step 20000: 0.178992
Minibatch accuracy: 93.8%
Validation accuracy: 90.0%
Minibatch loss at step 24000: 0.124309
Minibatch accuracy: 93.8%
Validation accuracy: 90.2%
Minibatch loss at step 28000: 0.586971
Minibatch accuracy: 81.2%
Validation accuracy: 90.5%
Minibatch loss at step 32000: 0.165207
Minibatch accuracy: 93.8%
Validation accuracy: 89.8%
Test accuracy: 95.4%


<pre> Cool 95.4 % ----Cheers !!!!! </pre>